[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jburgy/blog/blob/main/linprog/allocation_ampl.ipynb)

In [ ]:
%pip install amplpy ampl-module-base ampl-module-highs --extra-index-url https://pypi.ampl.com

In [ ]:
import numpy as np
import pandas as pd
from amplpy import ampl_notebook
from google.colab import userdata

ampl = ampl_notebook(license_uuid=userdata.get("AMPL_CE_LICENSE"))

In [ ]:
%%ampl_eval
set STOCKS;
set ACCOUNTS = {1..2};
set SECTORS;
# see https://groups.google.com/g/ampl/c/ULGck_3EQOM/m/yHvDLokzBQAJ
set STOCKS_IN_SECTOR {SECTORS} within STOCKS;

param base {ACCOUNTS};
param market_value {STOCKS};
param sign {s in STOCKS} = if market_value[s] >= 0 then 1 else -1;
param volume {STOCKS};

var X {s in STOCKS, ACCOUNTS}
          >= <<0; 1,0>> market_value[s],<= <<0; 0,1>> market_value[s];
var AbsX {STOCKS, ACCOUNTS} >= 0;
var GMV >= 0;
var Concentration {STOCKS};
var SectorMV {SECTORS};
var SectorNMV {SECTORS} >= 0;
var SectorExcess {SECTORS};

subject to Completeness {s in STOCKS}:
          sum {a in ACCOUNTS} X[s, a] = market_value[s];

subject to AbsXLevel {s in STOCKS, a in ACCOUNTS}:
          AbsX[s, a] = sign[s] * X[s, a];

subject to GMVLevel:
          GMV = sum {s in STOCKS} AbsX[s, 1];

subject to ConcentrationLevel {s in STOCKS}:
          Concentration[s] = AbsX[s, 1] - 0.05 * GMV;

subject to SectorMVLevel {sector in SECTORS}:
          SectorMV[sector] = sum {s in STOCKS_IN_SECTOR[sector]} X[s, 1];

subject to SectorNMVDomain {sector in SECTORS}:
          SectorNMV[sector] >= <<0; -1,1>> SectorMV[sector];

subject to SectorExcessLevel {sector in SECTORS}:
          SectorExcess[sector] = SectorNMV[sector] - 0.2 * GMV;

minimize TotalCost:
          sum {s in STOCKS, a in ACCOUNTS} base[a] * AbsX[s, a]
          + sum {s in STOCKS} <<volume[s]; 0,0.2>> AbsX[s, 1]
          + sum {s in STOCKS} <<0; 0,0.1>> Concentration[s]
          + sum {sector in SECTORS} <<0; 0,0.2>> SectorExcess[sector];

In [ ]:
π = pd.read_csv(
    "https://raw.githubusercontent.com/jburgy/jupyter"
    "/refs/heads/main/content/data/portfolio.csv",
    index_col=0,
)
# Fill-in blank ticker (ESC GCI LIBERTY INC SR COMMON STOCK)
π.rename(index={np.nan: "DUMMY"}, inplace=True)
π["market_value"] = π["Shares"] * π["Price"]
π["volume"] = π["Volume"] * π["Price"]

ampl.set_data(π[["market_value", "volume"]], set_name="STOCKS")
ampl.param["base"] = {1: 0.05, 2: 0.06}

stocks_in_sector = π.groupby("Sector").groups
ampl.set["SECTORS"] = stocks_in_sector.keys()
ampl.set["STOCKS_IN_SECTOR"] = stocks_in_sector

ampl.option["solver"] = "highs"
ampl.option["highs_options"] = {"outlev": 1}
ampl.solve()
assert ampl.solve_result == "solved"

In [ ]:
X = ampl.get_variable("X").get_values().to_pandas()
π.assign(
    account_1=X.loc[(π.index, 1), "X.val"].to_numpy(),
    account_2=X.loc[(π.index, 2), "X.val"].to_numpy(),
)